# Spatial Modulation , A deep learning based approach for maximising channel capacity for a Massive MIMO system for 5G communication application.

In [0]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import pandas as pd 
import numpy as np
from torch.utils.data.dataset import Dataset




In [0]:
# dataset 
dataset_dir='/content/Completedataset.csv'

class CustomDataset(Dataset):
  def __init__(self,csv_path):
        self.to_tensor = transforms.ToTensor()
        self.col_Names=["Bits Number","no of antennas","SNR","BER"]
        df=pd.read_csv(csv_path,names=self.col_Names)
        self.labels = np.asarray(df.iloc[:,3])
        self.inputbits =np.asarray(df.iloc[:,0])
        self.inputantennas =np.asarray(df.iloc[:,1])
        self.inputSNR =np.asarray(df.iloc[:,2])
        self.data_len=len(df.index)

  def __len__(self):
        return self.data_len

  def __getitem__(self, index):
        data= self.inputbits[index]
        data1=self.inputantennas[index]
        data2=self.inputSNR[index]
        data=np.array([data,data1,data2])
        label =np.array(self.labels[index])
        return torch.from_numpy(data),torch.from_numpy(label)


dataset=CustomDataset(dataset_dir)
print(dataset.__getitem__(9)[0].size())

#print(dataframe.head())new_a = a[np.newaxis, :]
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset,[train_size, test_size])

#print(train_dataset[0])



torch.Size([3])


In [0]:
# Hyper-parameters 
input_size = 3
output_size=1
hidden_size = 30
num_classes = 1
num_epochs = 1500
batch_size = 50
learning_rate = 0.05

# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,batch_size=batch_size,shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset,batch_size=batch_size,shuffle=True)



# Fully connected neural network with one hidden layer
class Spatialmodulation(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(Spatialmodulation, self).__init__()
        self.fc1 = nn.Linear(input_size,hidden_size) 
        self.bc1 = nn.BatchNorm1d(hidden_size)
        #self.drp1= nn.Dropout(0.5)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size,hidden_size)  
        self.bc2 = nn.BatchNorm1d(hidden_size)
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(hidden_size,hidden_size)  
        self.bc3 = nn.BatchNorm1d(hidden_size)
        self.relu3 = nn.ReLU()
        self.fc9 = nn.Linear(hidden_size,hidden_size)  
        self.bc9 = nn.BatchNorm1d(hidden_size)
        self.relu9 = nn.ReLU()
        self.fc4 = nn.Linear(hidden_size,output_size)  
        self.bc4 = nn.BatchNorm1d(output_size)
        #self.drp2= nn.Dropout(0.5)
        self.relu4 = nn.ReLU()

        self.fc5 = nn.Linear(output_size,num_classes) 
        self.relu5 = nn.ReLU()
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        out = self.relu2(out)
        out = self.fc3(out)
        out = self.relu3(out)
        out = self.fc9(out)
        out = self.relu9(out)
        out = self.fc4(out)
        out = self.relu4(out)
        out = self.fc5(out)
        out = self.relu5(out)
        return out

model = Spatialmodulation(input_size, hidden_size, num_classes)

print(model)

Spatialmodulation(
  (fc1): Linear(in_features=3, out_features=30, bias=True)
  (bc1): BatchNorm1d(30, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU()
  (fc2): Linear(in_features=30, out_features=30, bias=True)
  (bc2): BatchNorm1d(30, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu2): ReLU()
  (fc3): Linear(in_features=30, out_features=30, bias=True)
  (bc3): BatchNorm1d(30, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu3): ReLU()
  (fc9): Linear(in_features=30, out_features=30, bias=True)
  (bc9): BatchNorm1d(30, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu9): ReLU()
  (fc4): Linear(in_features=30, out_features=1, bias=True)
  (bc4): BatchNorm1d(1, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu4): ReLU()
  (fc5): Linear(in_features=1, out_features=1, bias=True)
  (relu5): ReLU()
)


In [0]:
# Loss and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(),lr=learning_rate)  

In [0]:
# Train the model
total_step = len(train_loader)
running_loss=0.0
loss_values=[]
for epoch in range(num_epochs):
    for i, (data,labels) in enumerate(train_loader):        
        # Forward pass
        labels=labels[:,np.newaxis]
        outputs = model(data.float())
        loss = criterion(outputs, labels.float())
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        #loss_values.append(loss.item())
        print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, i+1, total_step, loss.item()))        
 
        if (i+1) % batch_size == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

torch.save(model.state_dict(), 'model.ckpt')

Streaming output truncated to the last 5000 lines.
Epoch [1384/1500], Step [33/43], Loss: 0.0549
Epoch [1384/1500], Step [34/43], Loss: 0.0614
Epoch [1384/1500], Step [35/43], Loss: 0.0687
Epoch [1384/1500], Step [36/43], Loss: 0.0575
Epoch [1384/1500], Step [37/43], Loss: 0.0614
Epoch [1384/1500], Step [38/43], Loss: 0.0557
Epoch [1384/1500], Step [39/43], Loss: 0.0762
Epoch [1384/1500], Step [40/43], Loss: 0.0591
Epoch [1384/1500], Step [41/43], Loss: 0.0587
Epoch [1384/1500], Step [42/43], Loss: 0.0779
Epoch [1384/1500], Step [43/43], Loss: 0.0544
Epoch [1385/1500], Step [1/43], Loss: 0.0486
Epoch [1385/1500], Step [2/43], Loss: 0.0591
Epoch [1385/1500], Step [3/43], Loss: 0.0560
Epoch [1385/1500], Step [4/43], Loss: 0.0596
Epoch [1385/1500], Step [5/43], Loss: 0.0764
Epoch [1385/1500], Step [6/43], Loss: 0.0505
Epoch [1385/1500], Step [7/43], Loss: 0.0789
Epoch [1385/1500], Step [8/43], Loss: 0.0789
Epoch [1385/1500], Step [9/43], Loss: 0.0633
Epoch [1385/1500], Step [10/43], Loss:

In [0]:
model = Spatialmodulation(input_size, hidden_size, num_classes)
model.load_state_dict(torch.load('model.ckpt'))
model.eval()

# Test the model
# In test phase, we don't need to compute gradients (for memory efficiency)
with torch.no_grad():
    for (data,labels) in test_loader:
        #data=transforms.ToTensor(data)
        labels=labels[:,np.newaxis]
        outputs = model(data.float())
        loss = criterion(outputs,labels.float())
        print ('Output')



TypeError: ignored